# Weighted ALS from Hu, Koren and Volinksky's paper

http://yifanhu.net/PUB/cf.pdf

In [1]:
import os
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix, diags
from scipy.linalg import cho_solve, cho_factor, solve
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, accuracy_score, roc_curve, roc_auc_score
from sklearn.utils.extmath import safe_sparse_dot
import time

%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
lastfm_file = "/Users/timwee/projects/datasets/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv"

In [4]:
user_to_idx = {}
artist_to_idx = {}
users, artists, num_played = [], [], []
with open(lastfm_file, 'r') as f:
    for line in f:
        tup = line.strip().split("\t")
        usr, artist, cur_num_played = tup[0], tup[1], float(tup[3])
        if usr not in user_to_idx:
            user_to_idx[usr] = len(user_to_idx)
        if artist not in artist_to_idx:
            artist_to_idx[artist] = len(artist_to_idx)
        users.append(user_to_idx[usr])
        artists.append(artist_to_idx[artist])
        num_played.append(cur_num_played)

In [5]:
#del user_to_idx, artist_to_idx

## hyperparams and initialization

In [35]:
num_factors = 20
alpha = 40
epsilon = 1e-7
regularization = 1e-2

In [36]:
1 + alpha * np.log(1 + 0/epsilon), alpha * np.log(1 + 0/epsilon)

(1.0, 0.0)

In [37]:
# we don't add the 1 in the confidence formula, bec. we will subtract it out later anyway
#   This lets us keep the sparse matrix, otherwise 0 becomes 1
# just remember to add 1 when calculating b when solving the linear equation to C
confidence_vals = alpha * np.log(1 + np.array(num_played) / epsilon)

In [38]:
item_users_mat = coo_matrix((confidence_vals, (artists, users))).tocsr()        

In [39]:
user_items_mat = item_users_mat.T

In [40]:
num_users, num_items = user_items_mat.shape

In [41]:
import math

def bm25_weight(X, K1=100, B=0.8):
    """ Weighs each row of a sparse matrix X  by BM25 weighting """
    # calculate idf per term (user)
    X = coo_matrix(X)

    N = float(X.shape[0])
    idf = np.log(N / (1 + np.bincount(X.col)))

    # calculate length_norm per document (artist)
    row_sums = np.ravel(X.sum(axis=1))
    average_length = row_sums.mean()
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    X.data = X.data * (K1 + 1.0) / (K1 * length_norm[X.row] + X.data) * idf[X.col]
    return X

In [42]:
bm25_item_users_mat = bm25_weight(item_users_mat, K1=100, B=0.8)

In [45]:
user_factors = np.random.randn(num_users, num_factors) * 0.1
item_factors = np.random.randn(num_items, num_factors) * 0.1

## bench one row

For each row $x_u$ (user or item) we solve a linear equation. 
\begin{align}
x_u = (Y^TC^uY + \lambda I)^{-1} Y^TC^up(u)
\end{align}

- $C^{u}$ is a diagonal matrix where $C_{ii}^{u} = c_{ui}$, the "confidence" value of the user with the item.
- $Y$ is the item factors matrix (with dimensions **num_items X num_factors**)
- $\lambda$ is the regularization parameter
- $p(u)$ is the vector of preferences for user $u$. (this is the binarized values from the rating matrix.


#### confidence value and preferences

Hu, et. al computed $c_{ui}$ in 2 different ways:
1. $c_{ui} = 1 + \alpha r_{ui}$
    - $\alpha$ was set to 40 in the paper.
2. $c_{ui} = 1 + \alpha \log{({1 + r_{ui}} / \epsilon)}$


#### Rewrite of equation for faster runtime
We can rewrite $Y^TC^uY$ to avoid the two big matrix multiplies with $Y$ everytime for each user:
$$(Y^TC^uY) = (Y^T(I + C^u - I)Y) = Y^TY + Y^T(C^u - I)Y\\$$

This means that we can compute $Y^TY$ once and reuse it.

**Note:** This is mostly bec. of $C^u$ having values for all entries (a min value of 1 because the confidence formulas above have a $+ 1$). Since we don't add this $1$ to our confidence values when we form the matrix, we can directly compute $(Y^TC^uY)$

#### solving for $x_u$ 
To solve for $x_u$, we treat the first part of the equation above as the inverse, and the latter part as $b$, $x_u = A^{-1}b$, and pass it to a linear equations solver. 

Note that A is hermitian/symmetric and positive definite, so it is eligible to use a Cholesky decomposition solver.
\begin{align}
A = Y^TC^uY + \lambda I\\
A = Y^TY + Y^T(C^u - I)Y + \lambda I\\
b = Y^TC^up(u)\\
\end{align}



### for user

In [46]:
# grab a single row
usr_idx = 0

##### compute A's 3 terms
1. $Y^T(C^u - I)Y$
    - since we didn't add a 1 for the initial confidence matrix, we don't have to worry about the $I$,  so this becomes $Y^T(C^u)Y$
    - $C^u$ is a diagonal matrix with values on the diagonal (i,i) corresponding to confidence values between user $u$, and item $i$ 
    - since $C^u$ is a diagonal matrix, and $Y^T$ is dense, we don't have to do a dot product
        - multiplying a dense matrix $T$ by a diagonal matrix $D$ just multiplies each row or column $i$ of $T$ by the $(i, i)$ element in D.
        - use broadcasting instead. See https://stackoverflow.com/questions/13272453/multiplying-numpy-scipy-sparse-and-dense-matrices-efficiently
2. $Y^TY$
3. $\lambda I$



In [47]:
Y = item_factors
Yt = Y.T
YtY = np.dot(Yt, Y) # we don't need this as noted above

In [48]:
def YtCuIY_mask(Cu, Y, usr_idx=0, subtract_identity=False):
    """
    Remove zero elements for speed
    (Yt (Cu - I) Y).shape should be (num_factors, num_factors)
    Yt.shape == (num_factors, num_items)
    Cu.shape == diag(num_items, num_items)
    Y.shape == (num_items, num_factors)
    """
    if subtract_identity: 
        Cu = Cu.copy()
        Cu -= 1
        Cu[Cu < epsilon] = 0.0
    mask = Cu.nonzero()[0]#np.flatnonzero(Cu) # Cu.ravel().nonzero()[0]
    Cu_masked = Cu[mask]
    Y_masked = Y[mask,:]
    CuY = Cu_masked[:,None] * Y_masked # broadcast
    return Y_masked.T.dot(CuY)

In [49]:
def YtCuIY(Cu, Y, usr_idx=0, subtract_identity=False):
    """
    (Yt (Cu - I) Y).shape should be (num_factors, num_factors)
    Yt.shape == (num_factors, num_items)
    Cu.shape == diag(num_items, num_items)
    Y.shape == (num_items, num_factors)
    """
    if subtract_identity:
        Cu = Cu.copy()
        Cu -= 1
        Cu[Cu < epsilon] = 0.0
    CuY = Cu[:,None] * Y # broadcasting
    return Y.T.dot(CuY)

In [50]:
# Runs out of memory
def YtCuIY_nobroadcast(Cu, Y, usr_idx=0, subtract_identity=False):
    """
    (Yt (Cu - I) Y).shape should be (num_factors, num_factors)
    Yt.shape == (num_factors, num_items)
    Cu.shape == diag(num_items, num_items)
    Y.shape == (num_items, num_factors)
    """
    confidence_vals = Cu
    if subtract_identity:
        confidence_vals = Cu.copy()
        confidence_vals -= 1
        confidence_vals[confidence_vals < epsilon] = 0.0
    Cu = sparse.diags(confidence_vals, [0])
    return Y.T.dot(Cu).dot(Y)

In [51]:
Cu = user_items_mat[0,:].toarray().ravel()

In [52]:
%timeit YtCuIY_mask(Cu, Y)
%timeit YtCuIY(Cu, Y)

1000 loops, best of 3: 1.13 ms per loop
100 loops, best of 3: 12.7 ms per loop


In [53]:
# runs out of memory
#%timeit YtCuIY_nobroadcast(user_items_mat, Y)

In [54]:
np.allclose(YtCuIY_mask(Cu, Y), YtCuIY(Cu, Y))

True

##### compute $b = Y^TC^up(u)$

Note that $p(u)$, a binarized (0/1) diagonal matrix is implicitly computed already (non-zero inside $C^u$)

In [61]:
# b.shape == (num_factors, num_factors)
def compute_b(Cu, Y):
    """
    Cu is a 1-d array of confidence values for a particular usr u, with all items i
    
    Expected shapes:
    Cu.shape == (num_items,)
    Y.shape == (num_items, num_factors)
    """
    mask = Cu.copy()
    mask[mask > 0] = 1.0
    return (Y.T * ((Cu + 1.0) * mask)).sum(axis=1) # broadcast

In [62]:
mask = Cu.copy()
mask[mask > 0] = 1.0
masked_Cu = ((Cu + 1.0) * mask)
Cu.nonzero()[0], masked_Cu.nonzero()[0], Cu, masked_Cu

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]),
 array([ 951.41015625,  924.81006422,  916.68606053, ...,    0.        ,
           0.        ,    0.        ]),
 array([ 952.41015625,  925.81006422,  917.68606053, ...,    0.        ,
           0.        ,    0.        ]))

In [63]:
# b.shape == (num_factors, num_factors)
def compute_b_masked(Cu, Y):
    """
    Cu is a 1-d array of confidence values for a particular usr u, with all items i
    
    Expected shapes:
    Cu.shape == (num_items,)
    Y.shape == (num_items, num_factors)
    """
    mask = Cu.nonzero()[0]
    Cu_masked = Cu[mask]
    Y_masked = Y[mask,:]
    return (Y_masked.T * (Cu_masked + 1.0)).sum(axis=1) # broadcast

In [64]:
Y.T.shape, Cu.shape

((20, 160168), (160168,))

In [65]:
(Y.T * (Cu+1.0)).shape, (Y.T * (Cu+1.0)).sum(axis=1).shape

((20, 160168), (20,))

In [66]:
%timeit compute_b(Cu, Y)
%timeit compute_b_masked(Cu, Y)

100 loops, best of 3: 11.8 ms per loop
1000 loops, best of 3: 1 ms per loop


In [71]:
compute_b(Cu, Y), compute_b_masked(Cu, Y)

(array([  555.70413905,   138.55308784,   516.93944448,   -37.26347124,
           54.81983999,   290.06774156,  -145.58340345,  -249.89837671,
          454.00189552,  -584.38586968,   317.7771436 ,  -103.78444433,
          570.67511458,   191.91131591,  -119.64341522,   703.08735548,
          478.76139563,   471.03715306,   434.97177168, -1320.59185645]),
 array([  555.70413905,   138.55308784,   516.93944448,   -37.26347124,
           54.81983999,   290.06774156,  -145.58340345,  -249.89837671,
          454.00189552,  -584.38586968,   317.7771436 ,  -103.78444433,
          570.67511458,   191.91131591,  -119.64341522,   703.08735548,
          478.76139563,   471.03715306,   434.97177168, -1320.59185645]))

In [72]:
A = YtCuIY_mask(Cu, Y)
b = compute_b_masked(Cu, Y)
A.shape, b.shape

((20, 20), (20,))

In [73]:
c = cho_factor(A)
x = cho_solve(c,b)
x.shape, x

((20,), array([-0.02154763, -0.19682673,  1.34144086,  0.48305388,  1.84935232,
         0.80751548, -1.37784982, -0.21907132,  4.93612896, -1.90084841,
         0.98078027, -0.05312595,  2.2741765 ,  0.54577234,  0.92954333,
         2.34194063,  0.22752593,  3.4109749 ,  1.58488865, -5.05691554]))

In [74]:
lambda_I = regularization * np.eye(num_factors) # + YtY
def use_cho_solve(Cu, Y, lambda_I):
    A = YtCuIY_mask(Cu, Y) + lambda_I
    b = compute_b_masked(Cu, Y)
    c = cho_factor(A)
    x = cho_solve(c,b)
    return x

In [75]:
def gen_solve(Cu, Y, lambda_I):
    A = YtCuIY_mask(Cu, Y) + lambda_I
    b = compute_b_masked(Cu, Y)
    return solve(A, b)

In [76]:
use_cho_solve(Cu, Y, lambda_I).shape

(20,)

In [77]:
gen_solve(Cu, Y, lambda_I).shape

(20,)

In [78]:
%timeit use_cho_solve(Cu, Y, lambda_I)

100 loops, best of 3: 2.31 ms per loop


In [79]:
%timeit gen_solve(Cu, Y, lambda_I)

100 loops, best of 3: 2.34 ms per loop


# Putting it All Together

In [80]:
Cui = user_items_mat
Ciu = item_users_mat
lambda_I = regularization * np.eye(num_factors) # + YtY

In [81]:
def single_pass(Cui, Y, lambda_I, num_u, num_factors):
    result = np.zeros((num_u, num_factors))
    for idx, Cu in enumerate(Cui):
        result[idx] = use_cho_solve(Cu.toarray().ravel(), Y, lambda_I)
    return result

In [82]:
start = time.time()
single_pass(Cui=user_items_mat, Y=item_factors, lambda_I=lambda_I, num_u=num_users, num_factors=num_factors)
time.time() - start

971.4727640151978

In [83]:
start = time.time()
single_pass(Cui=item_users_mat, Y=user_factors, lambda_I=lambda_I, \
            num_u=num_items, num_factors=num_factors)
time.time() - start

1044.4756731987

In [84]:
def fit_weighted_als(user_items_mat, num_factors=15, num_iters=10):
    num_users, num_items = user_items_mat.shape
    user_factors = np.random.randn(num_users, num_factors) * 0.1
    item_factors = np.random.randn(num_items, num_factors) * 0.1
    
    item_users_mat = user_items_mat.T
    lambda_I = regularization * np.eye(num_factors)
    for num_iter in range(num_iters):
        # fit users
        start = time.time()
        user_factors = single_pass(Cui=user_items_mat, Y=item_factors, \
                                   lambda_I=lambda_I, num_u=num_users, \
                                   num_factors=num_factors)
        item_factors = single_pass(Cui=item_users_mat, Y=user_factors, \
                                   lambda_I=lambda_I, num_u=num_items, \
                                   num_factors=num_factors)
        print("finished iteration %i in %s" % (num_iter, time.time() - start))
    return user_factors, item_factors

In [85]:
start = time.time()
user_factors, item_factors = fit_weighted_als(user_items_mat, num_factors=25, num_iters=10)
end = time.time()
end - start

finished iteration 0 in 2174.3241670131683
finished iteration 1 in 1990.1240990161896
finished iteration 2 in 2052.0628390312195
finished iteration 3 in 1882.5355219841003
finished iteration 4 in 1796.1762628555298
finished iteration 5 in 1856.93275308609
finished iteration 6 in 1850.7348458766937
finished iteration 7 in 1773.3783450126648
finished iteration 8 in 1835.5745210647583
finished iteration 9 in 1833.4666879177094


19045.797111034393

# use sparse rep

In [86]:
def nonzeros(m, row):
    """ returns the non zeroes of a row in csr_matrix """
    for index in range(m.indptr[row], m.indptr[row+1]):
        yield m.indices[index], m.data[index]

def user_linear_equation(Y, YtY, Cui, u, regularization, n_factors):
    """
    YtY.shape == (n_factors, n_factors)
    """
    # This is from orig paper:
    #  Note that the -1 here is inverse of the matrix, not raising to power
    # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
    # solve for w* = A^-1 * b
    # A is YtCuY + regularization * I
    # b is YtCuPu

    ### This is the equation for "optimized version" for computing YtY outside the loop once
    # YtCuY + regularization * I = YtY + regularization * I + Yt(Cu-I)Y

    # accumulate YtCuY + regularization*I in A
    A = YtY + regularization * np.eye(n_factors)

    # accumulate YtCuPu in b
    # Pu is just the binarized rating/indicator variable
    b = np.zeros(n_factors)

    for i, confidence in nonzeros(Cui, u):
        # for user, this is the nonzero items
        # for items, nonzero users
        factor = Y[i]
        # Yt(Cu-I)Y - last term in "optimized" version
        A += (confidence - 1) * np.outer(factor, factor)
        # YtCuPu, factor is Yt, Pu is indicator variable taken care of by loop
        b += confidence * factor
    return A, b

def user_factor(Y, YtY, Cui, u, regularization, n_factors):
    # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
    A, b = user_linear_equation(Y, YtY, Cui, u, regularization, n_factors)
    return np.linalg.solve(A, b)


def least_squares(Cui, X, Y, regularization, num_threads=0):
    """ For each user in Cui, calculate factors Xu for them
    using least squares on Y.

    For fitting users:
    X is user factors (num_users, n_factors)
    Y is item factors (num_items, n_factors)

    Note: this is at least 10 times slower than the cython version included
    here.
    """
    users, n_factors = X.shape
    # shape - (num_factors, num_factors)
    YtY = Y.T.dot(Y)

    for u in range(users):
        X[u] = user_factor(Y, YtY, Cui, u, regularization, n_factors)
        
def fit_alternating_least_squares(item_users, num_factors=20, dtype=np.float32, \
                                 num_iters=5, regularization=0.01, num_threads=0):
    Ciu, Cui = item_users.tocsr(), item_users.T.tocsr()
    items, users = Ciu.shape

    # Initialize the variables randomly if they haven't already been set
    user_factors = np.random.rand(users, num_factors).astype(dtype) * 0.01
    item_factors = np.random.rand(items, num_factors).astype(dtype) * 0.01

    # alternate between learning the user_factors from the item_factors and vice-versa
    for iteration in range(num_iters):
        s = time.time()
        least_squares(Cui, user_factors, item_factors, regularization,
               num_threads=num_threads)
        least_squares(Ciu, item_factors, user_factors, regularization,
               num_threads=num_threads)
        print("finished iteration %i in %s", iteration, time.time() - s)

#         if self.calculate_training_loss:
#             loss = _als.calculate_loss(Cui, self.user_factors, self.item_factors,
#                                        self.regularization, num_threads=self.num_threads)
#             log.debug("loss at iteration %i is %s", iteration, loss)
    return user_factors, item_factors


In [87]:
start = time.time()
user_factors, item_factors = fit_alternating_least_squares(bm25_item_users_mat)
time.time() - start

finished iteration %i in %s 0 507.07953810691833
finished iteration %i in %s 1 507.20902609825134
finished iteration %i in %s 2 507.10115599632263
finished iteration %i in %s 3 508.3597388267517
finished iteration %i in %s 4 508.0019760131836


2545.6993560791016